# train  LG GB SVM-BRF models on imbalanced/balanced ClinVar data with 29 features

Method:  
Sklearn data preprocessing pipeline,   
joblib save model, save preprocessor

In [5]:
import gzip
import pandas as pd
import numpy as np
from joblib import dump, load


# extract  features

In [6]:



"""1.for outside CSQ: get feature names """

features = ['CLNSIG','CADD','AF_ESP','AF_EXAC','AF_TGP','Frq','GNOMADAF','GNOMADAF_popmax','Hom','ORIGIN','SPIDEX','SWEGENAF'] ## keep CLNSIG

## get features' value start site
feature_value_start={}
for i in features:
    feature_value_start[i]= (len(i) +1)



"""2.for outside CSQ,function: extract features' value """
def extract_feature(features,feature_value_start,info):
    features_outside = []

    if any('SWEGENAAC_Hom=' in str for str in info):             # remove 'SWEGENAAC_Hom=' in advance, avoid mess 'HOM' later
        match = [s for s in info if 'SWEGENAAC_Hom=' in s]       # match is a list, [0] to get str item
        try:
            info.remove(match[0])
        except ValueError:
            pass


    for i in features:
        prefix = (i+'=')   
        if any (prefix in str for str in info):                  # check for the presence 'score' in INFO:
            matching = [s for s in info if prefix in s]          # get the item containing 'score':
            value_start =  feature_value_start[i]          
            features_outside.append(matching[0][value_start:])   # get the value
        else:
            features_outside.append('')                          # if not exist, add empty

    return features_outside                                      # 2D list




"""3.for CSQ: get 81 fields names"""

csq_name='Allele|Consequence|IMPACT|SYMBOL|Gene|Feature_type|Feature|BIOTYPE|EXON|INTRON|HGVSc|HGVSp|cDNA_position|CDS_position|Protein_position|Amino_acids|Codons|Existing_variation|DISTANCE|STRAND|FLAGS|SYMBOL_SOURCE|HGNC_ID|CANONICAL|TSL|APPRIS|CCDS|ENSP|SWISSPROT|TREMBL|UNIPARC|UNIPROT_ISOFORM|REFSEQ_MATCH|SOURCE|REFSEQ_OFFSET|GIVEN_REF|USED_REF|BAM_EDIT|SIFT|PolyPhen|DOMAINS|HGVS_OFFSET|MOTIF_NAME|MOTIF_POS|HIGH_INF_POS|MOTIF_SCORE_CHANGE|TRANSCRIPTION_FACTORS|MES-NCSS_downstream_acceptor|MES-NCSS_downstream_donor|MES-NCSS_upstream_acceptor|MES-NCSS_upstream_donor|MES-SWA_acceptor_alt|MES-SWA_acceptor_diff|MES-SWA_acceptor_ref|MES-SWA_acceptor_ref_comp|MES-SWA_donor_alt|MES-SWA_donor_diff|MES-SWA_donor_ref|MES-SWA_donor_ref_comp|MaxEntScan_alt|MaxEntScan_diff|MaxEntScan_ref|GERP++_NR|GERP++_RS|REVEL_rankscore|REVEL_score|phastCons100way_vertebrate|phyloP100way_vertebrate|rs_dbSNP150|LoFtool|pLI_gene_value|SpliceAI_pred_DP_AG|SpliceAI_pred_DP_AL|SpliceAI_pred_DP_DG|SpliceAI_pred_DP_DL|SpliceAI_pred_DS_AG|SpliceAI_pred_DS_AL|SpliceAI_pred_DS_DG|SpliceAI_pred_DS_DL|SpliceAI_pred_SYMBOL|genomic_superdups_frac_match'
csq_name_split=csq_name.split('|')

## get two fields' index
con_idx = csq_name_split.index('Consequence') 
CANONICALidx = csq_name_split.index('CANONICAL')




"""4. for CSQ: give Consequence order """

with open('variant_consequences.txt','r') as f:            # get the 'variant_consequences order' lst: descending severity
    order_conseq=[line.rstrip("\n") for line in f]         # remove the \n at right side of each line; lst

# give consequence order number
orderdic = {v:i for i,v in enumerate(order_conseq)}




"""5.for CSQ, function: split transcripts into fields, and clean Consequence cell"""

def split_CSQ_fields(trans_csq,order_conseq,con_idx):   

    for i in range(len(trans_csq)):
            split_trans = trans_csq[i].split('|')         # split each trans into fields as str
                                                                                                             
            cons_sub = split_trans[con_idx].split('&')    # in each trans: split `Consequence` cell inside
            cons_sub.sort(key=order_conseq.index)         # in each trans: order `Consequence` cell inside
            split_trans[con_idx] = cons_sub[0]            # in each trans: only keep the most severe `Consequence` in this cell
            
            trans_csq[i] = split_trans                    # 2D lst
    return trans_csq





"""6.for CSQ, function: pick one trans of each variant: `Conseqence` is most severe and `Canonical`==Yes """

def pick_CSQ_trans_final(trans_csq,orderdic,con_idx,CANONICALidx):

    ### order and pick one trans of each variant
    trans_csq.sort(key=lambda x:orderdic[x[con_idx]])     # in each variant: order trans by `Consequence` cell

    found = -1
    for i in range(len(trans_csq)):                       
        if(trans_csq[i][CANONICALidx] == "YES" and trans_csq[i][con_idx] == trans_csq[0][con_idx]):         
            found = i                                 
            break
    if(not found==-1):
        picked_trans=(trans_csq[found] )                  # in each variant: if any trans `Cons` is the most sever and `Canonical`=YES, pick this trans
    else:                                                 # if no any `Canonical`= yes, pick the fist most sever trans
        picked_trans=(trans_csq[0])

    return picked_trans





""" 7. main script: extract all features outside and inside CSQ"""

in_file = '/Users/nancy/Desktop/RS_projects/data/02_data_analysis/00_original_data/clinvar_221113.vcf.gz'
out_file = '/Users/nancy/Desktop/RS_projects/data/03_ML/01_featureV1/01_train/modelV2/01_ALLfeatures_extracted_addCLNSIG.csv'

 
head_line = float('inf')                                            #  as infinity integer

with gzip.open(in_file) as in_f:                                    # read compressed file
    with open(out_file,'w') as out_f:                               # write new file.csv
        
        ### write the head of features outside CSQ and CSQ
        out_f.write(';'.join(features) +';')                        
        out_f.write(';'.join(csq_name_split)+'\n')                  


        for i,line in enumerate(in_f,0):                           
            content = line.decode('utf8').rstrip('\n')             
            if content.startswith('#CHROM'):                        # find first line of rec 
                rec_head=content.split('\t')                        # rec head line: split str to list
                INFOidx = rec_head.index('INFO')                    # get INFO idx
                head_line = i


            if i > head_line:
                rec_lst=content.split('\t')                                                  # lst: get each variant line , split str

                ### extract features outside CSQ
                info = rec_lst[INFOidx].split(';')                                           # lst: get each INFO line               
                feature_line = extract_feature(features,feature_value_start,info)            # get each feature line

                ### extract CSQ
                trans_csq = rec_lst[INFOidx].split('CSQ=')[-1].split(";")[0].split(",")       # in each variant: get csq's transcripts , lst
                trans_csq = split_CSQ_fields(trans_csq,order_conseq,con_idx)                  # in each variant: splite each transcript into fields, and ordered by `Consequence``
                picked_trans = pick_CSQ_trans_final(trans_csq,orderdic,con_idx,CANONICALidx)  # in each variant: pick one transcript which `Conseqence` is most severe and `Canonical`==Yes """
                
                ### write down
                out_f.write(';'.join(feature_line) +';')           
                out_f.write(';'.join(picked_trans) +'\n')           # why ';' instead of ','--- since CLNSIG has'Likely_pathogenic,_low_penetrance' with ',' inside to mess the formate CSV


FileNotFoundError: [Errno 2] No such file or directory: '/Users/nancy/Desktop/RS_projects/data/03_ML/01_featureV1/01_train/modelV2/01_ALLfeatures_extracted_addCLNSIG.csv'

# Data preprocessing pipeline

In [7]:
df = pd.read_csv('/Users/nancy/Desktop/RS_projects/data/03_ML/01_train_model/01_featureV1/01_ALLfeatures_extracted_addCLNSIG.csv',sep=';',index_col=False)
df

/var/folders/q0/2p8kfxsn7s73349btf1_g07h0000gp/T/ipykernel_4968/3969921985.py:1: DtypeWarning: Columns (32,49,50,51,54,56,58,92) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/Users/nancy/Desktop/RS_projects/data/03_ML/01_train_model/01_featureV1/01_ALLfeatures_extracted_addCLNSIG.csv',sep=';',index_col=False)


,CLNSIG,CADD,AF_ESP,AF_EXAC,AF_TGP,Frq,GNOMADAF,GNOMADAF_popmax,Hom,ORIGIN,...,SpliceAI_pred_DP_AG,SpliceAI_pred_DP_AL,SpliceAI_pred_DP_DG,SpliceAI_pred_DP_DL,SpliceAI_pred_DS_AG,SpliceAI_pred_DS_AL,SpliceAI_pred_DS_DG,SpliceAI_pred_DS_DL,SpliceAI_pred_SYMBOL,genomic_superdups_frac_match
0,Uncertain_significance,26.600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,-30.0,11.0,10.0,-30.0,0.01,0.00,0.09,0.00,SAMD11,NaN
1,Likely_benign,13.420,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,7.0,-34.0,6.0,42.0,0.00,0.02,0.03,0.00,SAMD11,NaN
2,Likely_benign,31.000,NaN,NaN,NaN,0.00056,0.000414,0.000195,NaN,1.0,...,41.0,-47.0,-7.0,44.0,0.00,0.01,0.06,0.00,SAMD11,NaN
3,Uncertain_significance,28.200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,10.0,-47.0,34.0,-14.0,0.00,0.00,0.03,0.02,SAMD11,NaN
4,Likely_benign,11.380,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,-35.0,-50.0,12.0,24.0,0.00,0.00,0.01,0.00,SAMD11,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468907,Benign,7.587,NaN,NaN,NaN,0.00070,NaN,NaN,5.0,1.0,...,25.0,35.0,0.0,16.0,0.00,0.00,0.00,0.00,USP9Y,NaN
1468908,Uncertain_significance,23.800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1468909,Benign,11.200,0.00614,0.00589,0.00243,0.00671,NaN,NaN,48.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1468910,Uncertain_significance,21.600,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
print(df.min())

In [ ]:
df.columns

In [8]:
""" 1. choose featuresV1"""

featureV1 = ['CLNSIG','CADD', 'AF_TGP', 'Frq', 'IMPACT','GNOMADAF_popmax', 'Hom', 'ORIGIN',
       'SPIDEX', 'SWEGENAF', 'Consequence', 'BIOTYPE', 'SIFT',
       'PolyPhen', 'MES-SWA_acceptor_alt', 'MES-SWA_acceptor_diff',
       'MES-SWA_donor_alt', 'MES-SWA_donor_diff', 'MaxEntScan_alt',
       'MaxEntScan_diff', 'GERP++_RS', 'REVEL_score',
       'phastCons100way_vertebrate', 'phyloP100way_vertebrate', 'LoFtool',
       'pLI_gene_value', 'SpliceAI_pred_DS_AG', 'SpliceAI_pred_DS_AL',
       'SpliceAI_pred_DS_DG', 'SpliceAI_pred_DS_DL']         # keep CLNSIG,

df_1 = df[featureV1]



""" 2. only remove missing target """
# for some features with fewer missing
# drop_nan_features = ['CLNSIG','CADD','Consequence','IMPACT','BIOTYPE','MES-SWA_acceptor_alt','MES-SWA_acceptor_diff',
# 'MES-SWA_donor_alt','MES-SWA_donor_diff','pLI_gene_value']

drop_nan_features = ['CLNSIG']
df_1 = df_1.dropna(subset=drop_nan_features)

In [9]:
len(featureV1)  

30

In [10]:
""" 3. deal with target feature """
# """  CLNSIG: tansfer to 4 classes__ benign,pathogenic,uncertain,others
# NOTE:
# (only count name before |)   explanation-- https://www.ncbi.nlm.nih.gov/clinvar/docs/clinsig/#clinsig_scv

# 1. pathogenic: Pathogenic, Pathogenic/Likely_pathogenic, Likely_pathogenic
# 2. benign: Likely_benign, Benign, Benign/Likely_benign
# 3. uncertain: Uncertain_significance, 
# 4. others: Uncertain_risk_allele, risk_factor, protective, other, not_provided, Likely_risk_allele, drug_response, Conflicting_interpretations_of_pathogenicity, confers_sensitivity, associatio, Affects

df_1 = df_1.copy()
df_1['CLNSIG'] = df_1['CLNSIG'].replace(['Uncertain_significance|_risk_factor', 'Uncertain_significance|_other', 'Uncertain_significance|_drug_response','Uncertain_significance|_association','Uncertain_significance|_Affects','Uncertain_significance'], 'uncertain')
df_1['CLNSIG'] = df_1['CLNSIG'].replace(['Pathogenic|_risk_factor','Pathogenic|_protective','Pathogenic|_other','Pathogenic|_drug_response|_other','Pathogenic|_drug_response','Pathogenic|_confers_sensitivity','Pathogenic|_association','Pathogenic|_Affects','Pathogenic/Likely_risk_allele','Pathogenic/Likely_pathogenic|_risk_factor','Pathogenic/Likely_pathogenic|_other','Pathogenic/Likely_pathogenic|_drug_response','Pathogenic/Likely_pathogenic','Pathogenic','Likely_pathogenic|_risk_factor','Likely_pathogenic|_other','Likely_pathogenic|_drug_response','Likely_pathogenic|_association','Likely_pathogenic|_Affects','Likely_pathogenic,_low_penetrance','Likely_pathogenic'], 'pathogenic')
df_1['CLNSIG'] = df_1['CLNSIG'].replace(['Likely_benign|_risk_factor','Likely_benign|_other','Likely_benign|_drug_response|_other','Likely_benign','Benign|_risk_factor','Benign|_protective','Benign|_other','Benign|_drug_response','Benign|_confers_sensitivity','Benign|_association|_confers_sensitivity','Benign|_association','Benign/Likely_benign|_risk_factor','Benign/Likely_benign|_other|_risk_factor','Benign/Likely_benign|_other','Benign/Likely_benign|_drug_response|_other','Benign/Likely_benign|_drug_response','Benign/Likely_benign|_association','Benign/Likely_benign','Benign'], 'benign')
df_1['CLNSIG'] = df_1['CLNSIG'].replace(['Uncertain_risk_allele|_risk_factor','Uncertain_risk_allele','risk_factor','protective','protective|_risk_factor','other', 'not_provided','Likely_risk_allele','drug_response|_risk_factor','drug_response|_other','drug_response','Conflicting_interpretations_of_pathogenicity|_risk_factor','Conflicting_interpretations_of_pathogenicity|_protective','Conflicting_interpretations_of_pathogenicity|_other|_risk_factor','Conflicting_interpretations_of_pathogenicity|_other','Conflicting_interpretations_of_pathogenicity|_drug_response|_other','Conflicting_interpretations_of_pathogenicity|_drug_response','Conflicting_interpretations_of_pathogenicity|_association|_risk_factor','Conflicting_interpretations_of_pathogenicity|_association','Conflicting_interpretations_of_pathogenicity','confers_sensitivity','association|_risk_factor','association_not_found','association','Affects|_risk_factor','Affects|_association','Affects'], 'others')

# only keep benign & pathogenic
df_1 = df_1.loc[df_1['CLNSIG'].isin(['benign','pathogenic'])]

# label encoder
map_clnsig={'pathogenic':1,'benign':0}
df_1['CLNSIG']=df_1['CLNSIG'].map(lambda s: map_clnsig.get(s) if s in map_clnsig else s)
# df_1["CLNSIG"].unique()

In [11]:
df_1

,CLNSIG,CADD,AF_TGP,Frq,IMPACT,GNOMADAF_popmax,Hom,ORIGIN,SPIDEX,SWEGENAF,...,GERP++_RS,REVEL_score,phastCons100way_vertebrate,phyloP100way_vertebrate,LoFtool,pLI_gene_value,SpliceAI_pred_DS_AG,SpliceAI_pred_DS_AL,SpliceAI_pred_DS_DG,SpliceAI_pred_DS_DL
1,0,13.420,NaN,NaN,LOW,NaN,NaN,1.0,-1.030,NaN,...,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.02,0.03,0.00
2,0,31.000,NaN,0.00056,MODERATE,0.000195,NaN,1.0,-0.910,NaN,...,4.52,0.429,1.0,5.538,NaN,0.00,0.00,0.01,0.06,0.00
4,0,11.380,NaN,NaN,LOW,NaN,NaN,1.0,0.992,NaN,...,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.01,0.00
6,0,9.519,NaN,NaN,LOW,NaN,NaN,1.0,-1.049,NaN,...,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.05,0.17
8,0,5.182,NaN,NaN,LOW,NaN,NaN,1.0,-1.021,NaN,...,NaN,NaN,NaN,NaN,NaN,0.01,0.03,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1468905,0,4.853,NaN,0.18160,LOW,NaN,NaN,1.0,NaN,0.274000,...,NaN,NaN,NaN,NaN,NaN,0.31,0.00,0.00,0.00,0.00
1468906,1,26.500,NaN,0.00014,HIGH,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,1.00
1468907,0,7.587,NaN,0.00070,LOW,NaN,5.0,1.0,-0.088,NaN,...,NaN,NaN,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00
1468909,0,11.200,0.00243,0.00671,LOW,NaN,48.0,1.0,-1.796,0.001927,...,NaN,NaN,NaN,NaN,NaN,0.10,NaN,NaN,NaN,NaN


In [ ]:
""" 4. split data into train and test""" 

""" try to train twice, since imbalance classes

4.1 make balanced classes: undersampling the benign samples to make two sample uneven"""

# Undersampling: balance uneven benign and pathogenic quantity
# benign = df_1[df_1["CLNSIG"]==0]
# pathogenic = df_1[df_1["CLNSIG"]==1]
# benign = benign.sample(len(pathogenic),random_state=42)   # randomly extract benign same quantity with patho
# data = pd.concat( (benign,pathogenic),axis=0)

# # split
# from sklearn.model_selection import train_test_split

# X, y = data.iloc[:, 1:], data.iloc[:, 0]
# X_train, X_test, y_train, y_test =\
#     train_test_split(X, y, 
#                     test_size=0.3,
#                     random_state=42,
#                     stratify=y)

In [12]:
"""4.2  keep imbalance classes """

# split
from sklearn.model_selection import train_test_split

X, y = df_1.iloc[:, 1:], df_1.iloc[:, 0]
X_train, X_test, y_train, y_test =\
    train_test_split(X, y, 
                    test_size=0.3,
                    random_state=42,
                    stratify=y)

In [14]:
variant_consequences_order = './variant_consequences.txt'

In [ ]:
# old version: include remove missing data features
# 
# """ 5. features engineering: transformer"""
# ## only for training data, no care about target 'CLNSIG'

# from sklearn.compose import make_column_transformer
# from sklearn.impute import MissingIndicator, SimpleImputer
# from sklearn.preprocessing import OrdinalEncoder, StandardScaler
# from category_encoders import BinaryEncoder

# ## numerical feature group 1 : replace missing with mean;  scaling/standardization

# numeric_feature_1 = ['MaxEntScan_alt','MaxEntScan_diff','GERP++_RS',
#     'phastCons100way_vertebrate','phyloP100way_vertebrate']



# ## numerical feature group 2 : no missing left or/ replace missing with 0;  scaling/standardization
# numeric_feature_2 = ['CADD','AF_TGP', 'Frq', 'GNOMADAF_popmax', 'Hom',
#        'SWEGENAF','MES-SWA_acceptor_alt','MES-SWA_acceptor_diff','MES-SWA_donor_alt',
#        'MES-SWA_donor_diff','LoFtool','pLI_gene_value','SpliceAI_pred_DS_AL',
#        'SpliceAI_pred_DS_DG','SpliceAI_pred_DS_DL']



# ## numerical feature group 3: add missing indicator; replace missing with 0 and ; scaling/standardization
# numeric_feature_3 = ['SPIDEX','REVEL_score','SpliceAI_pred_DS_AG']



# ## categorical ordinal feature group 4: add missing indicator; no missing left or/ replace missing with 0; OrdinalEncoder
# categori_oridinal_feature_4 =['Consequence','IMPACT','SIFT','PolyPhen']

# # ordinal features' order: left to right will be from 0 to length-1
# with open(variant_consequences_order,'r') as f:             # get the 'variant_consequences order' lst: descending severity
#     order_conseq=[line.rstrip("\n") for line in f] 
#     order_conseq.reverse()

# ordinal_features = [
#     'Consequence',
#     'IMPACT',
#     'SIFT',
#     'PolyPhen']
# # 0 is the missing value will be replaced by
# ordinal_ordering = [
#     order_conseq,
#     ['MODIFIER','LOW','MODERATE','HIGH'],
#     [0,'tolerated_low_confidence','tolerated','deleterious_low_confidence','deleterious'],
#     [0,'unknown','benign','possibly_damaging','probably_damaging']
#     ]


# ## categorical nominal feature group 5: no missing left or/ replace missing with 0; BinaryEncoder
# categori_nominal_feature_5 = ['ORIGIN','BIOTYPE']  #,'BIOTYPE'


# from sklearn.pipeline import Pipeline, make_pipeline
# from sklearn.compose import ColumnTransformer, make_column_transformer

# numeric_feature_1_transformer =make_pipeline(
#     SimpleImputer(strategy='mean'),
#     StandardScaler()
# )

# numeric_feature_2_transformer =make_pipeline(
#     SimpleImputer(strategy='constant',fill_value=0),
#     StandardScaler()
# )

# numeric_feature_3_transformer =make_pipeline(
#     SimpleImputer(strategy='constant',fill_value=0),
#     StandardScaler()
# )

# categori_oridinal_feature_4_transformer =make_pipeline(
#     SimpleImputer(strategy='constant',fill_value=0), 
#     OrdinalEncoder(categories=ordinal_ordering)
# )

# categori_oridinal_feature_5_transformer =make_pipeline(
#     SimpleImputer(strategy='constant',fill_value=0),
#     BinaryEncoder()                                # add new columns, throw the original columns automaticly, 
# )


# preprocessor = make_column_transformer(
#     (numeric_feature_1_transformer, numeric_feature_1),
#     (numeric_feature_2_transformer, numeric_feature_2 ),
#     (MissingIndicator(), numeric_feature_3 + categori_oridinal_feature_4),   # add new missing indicators for feature group 3-4 without originial columns,  here: add 5 indicator columns in total
#     (numeric_feature_3_transformer, numeric_feature_3),  
#     (categori_oridinal_feature_4_transformer, categori_oridinal_feature_4),
#     (categori_oridinal_feature_5_transformer, categori_nominal_feature_5),   # here: 'ORIGIN': 6 new, 'BIOTYPE': 3 new
#     remainder='passthrough'

# )



In [15]:
""" 5. features engineering: transformer"""
## only for training data, no care about target 'CLNSIG', total 29 features

from sklearn.compose import make_column_transformer
from sklearn.impute import MissingIndicator, SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from category_encoders import BinaryEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_transformer

## numerical feature group 1 : replace missing with mean;  scaling/standardization

numeric_feature_1 = ['MES-SWA_acceptor_alt','MES-SWA_acceptor_diff','MES-SWA_donor_alt','MES-SWA_donor_diff','MaxEntScan_alt','MaxEntScan_diff','GERP++_RS',
    'phastCons100way_vertebrate','phyloP100way_vertebrate']



## numerical feature group 2 : replace missing with 0;  scaling/standardization
numeric_feature_2 = ['CADD','AF_TGP', 'Frq', 'GNOMADAF_popmax', 'Hom',
       'SWEGENAF','LoFtool','pLI_gene_value','SpliceAI_pred_DS_AL',
       'SpliceAI_pred_DS_DG','SpliceAI_pred_DS_DL']



## numerical feature group 3: add missing indicator; replace missing with 0 and ; scaling/standardization
numeric_feature_3 = ['SPIDEX','REVEL_score','SpliceAI_pred_DS_AG']



## categorical ordinal feature group 4: add missing indicator for ['SIFT','PolyPhen']; replace missing with 'missing_value' string; OrdinalEncoder
categori_oridinal_feature_4 =['Consequence','IMPACT','SIFT','PolyPhen']

# ordinal features' order: left to right will be from 0 to length-1
with open(variant_consequences_order,'r') as f:             # get the 'variant_consequences order' lst: descending severity
    order_conseq=[line.rstrip("\n") for line in f] 
    order_conseq.reverse()

ordinal_features = [
    'Consequence',
    'IMPACT',
    'SIFT',
    'PolyPhen']
# 0 is the missing value will be replaced by
ordinal_ordering = [
    order_conseq,
    ['MODIFIER','LOW','MODERATE','HIGH'],
    ['tolerated_low_confidence','tolerated','deleterious_low_confidence','deleterious'],
    ['unknown','benign','possibly_damaging','probably_damaging']
    ]


## categorical nominal feature group 5: replace missing with 0 in ['ORIGIN'], with ''missing_value' in ['BIOTYPE'];  BinaryEncoder
categori_nominal_feature_5 = ['ORIGIN','BIOTYPE']  #,'BIOTYPE'



numeric_feature_1_transformer =make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler()
)

numeric_feature_2_transformer =make_pipeline(
    SimpleImputer(strategy='constant',fill_value=0),
    StandardScaler()
)

numeric_feature_3_transformer =make_pipeline(
    SimpleImputer(strategy='constant',fill_value=0),
    StandardScaler()
)

categori_oridinal_feature_4_transformer =make_pipeline(
    SimpleImputer(strategy='constant',fill_value=None),  # If None, fill_value will be 0 when imputing numerical data and “missing_value” for strings
    OrdinalEncoder(categories=ordinal_ordering, handle_unknown='use_encoded_value', unknown_value = -1)       # the unknown 'missing_value' string will be enocoded as '-1'
)

categori_oridinal_feature_5_transformer =make_pipeline(
    SimpleImputer(strategy='constant',fill_value=None),  
    BinaryEncoder()                                # add new columns, throw the original columns automaticly, 
)


preprocessor = make_column_transformer(
    (numeric_feature_1_transformer, numeric_feature_1),
    (numeric_feature_2_transformer, numeric_feature_2 ),
    (MissingIndicator(), numeric_feature_3 + categori_oridinal_feature_4[2:]),   # add new missing indicators for feature3 + ['SIFT','PolyPhen'] without originial columns,  here: total add 5 indicator columns
    (numeric_feature_3_transformer, numeric_feature_3),  
    (categori_oridinal_feature_4_transformer, categori_oridinal_feature_4),
    (categori_oridinal_feature_5_transformer, categori_nominal_feature_5),   # here: 'ORIGIN': 6 new, 'BIOTYPE': 3 new
    # remainder='passthrough'                                                   # 

)



In [ ]:
from sklearn import set_config

set_config(display="diagram")

preprocessor

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['MES-SWA_acceptor_alt',
                                  'MES-SWA_acceptor_diff', 'MES-SWA_donor_alt',
                                  'MES-SWA_donor_diff', 'MaxEntScan_alt',
                                  'MaxEntScan_diff', 'GERP++_RS',
                                  'phastCons100way_vertebrate',
                                  'phyloP100way_vertebrate']),
                                ('pipeline-2',
                                 Pipeline(...
                                                                              'tolerated',
                                                                              'deleterious_low_confidence',
                                                                              'deleterious'],
                                                                             ['unknown',
                                                                              'benign',
                                                                              'possibly_damaging',
                                                                              'probably_damaging']],
                                                                 handle_unknown='use_encoded_value',
                                                                 unknown_value=-1))]),
                                 ['Consequence', 'IMPACT', 'SIFT', 'PolyPhen']),
                                ('pipeline-5',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='constant')),
                                                 ('binaryencoder',
                                                  BinaryEncoder())]),
                                 ['ORIGIN', 'BIOTYPE'])])

In [16]:
len(numeric_feature_1)+len(numeric_feature_2)+len(categori_oridinal_feature_4)+len(numeric_feature_3)+len(categori_nominal_feature_5)

29

# train model

## LG

In [17]:
""" 6. get model"""

# model: logstic regression 26s
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(max_iter=1500)   # max iteration to finish converage: max_iter=1500 , C =1


# Define pipeline: data preprocessing + model
pipeline = make_pipeline(preprocessor, logreg)      # preprocessor will only be used for X train automatically


# Train model on training data
pipeline.fit(X_train, y_train)

# Evaluate model on test data
accuracy_train_lr = pipeline.score(X_train, y_train)
accuracy_test_lr = pipeline.score(X_test, y_test)

print("LR Training set score: {:.6f}".format(accuracy_train_lr)) 
print("LR Test set score: {:.6f}".format(accuracy_test_lr))

LG Training set score: 0.980959
LG Test set score: 0.981063


In [ ]:
# # Save the preprocessor and model: for deal with imbalance
# dump(preprocessor, '/Users/nancy/Desktop/RS_projects/result/03_ML/01_train_model/01_featureV1/models/01_preprocessor.joblib')
# dump(logreg, '/Users/nancy/Desktop/RS_projects/result/03_ML/01_train_model/01_featureV1/models/01_LR_model.joblib')


['/Users/nancy/Desktop/RS_projects/result/03_ML/01_train_model/01_featureV1/models/01_LG_model.joblib']

In [20]:
# Save the preprocessor and model: for keep imbalance
dump(preprocessor, '/Users/nancy/Desktop/RS_projects/result/03_ML/01_train_model/01_featureV1/models/01_preprocessor_imbalance.joblib')
dump(logreg, '/Users/nancy/Desktop/RS_projects/result/03_ML/01_train_model/01_featureV1/models/01_LR_model_imbalance.joblib')

['/Users/nancy/Desktop/RS_projects/result/03_ML/01_train_model/01_featureV1/models/01_LR_model_imbalance.joblib']

## GB

In [19]:
## try another model: Gradient boosted : 5min

from sklearn.ensemble import GradientBoostingClassifier
gbrt = GradientBoostingClassifier(n_estimators=500,random_state=0)

# Define pipeline_2: data preprocessing + model
pipeline_2 = make_pipeline(preprocessor, gbrt)

# Train model on training data
pipeline_2.fit(X_train, y_train)

# Evaluate model on test data
accuracy_train_gb = pipeline_2.score(X_train, y_train)
accuracy_test_gb = pipeline_2.score(X_test, y_test)

print("GB Training set score: {:.6f}".format(accuracy_train_gb)) 
print("GB Test set score: {:.6f}".format(accuracy_test_gb))

GB Training set score: 0.989469
GB Test set score: 0.988763


In [21]:
# Save model: the same preprocessor already saved

dump(gbrt, '/Users/nancy/Desktop/RS_projects/result/03_ML/01_train_model/01_featureV1/models/01_GB_model_imbalance.joblib')

['/Users/nancy/Desktop/RS_projects/result/03_ML/01_train_model/01_featureV1/models/01_GB_model_imbalance.joblib']

## SVM RBF

In [22]:
## try SVM RBF:  very slow

from sklearn.svm import SVC


svc = SVC(probability=True)                  # By default, C=1 and gamma=1/n_features ; prprobability=True to make it has prob

# Define pipeline_3: data preprocessing + model
pipeline_3 = make_pipeline(preprocessor, svc)

# Train model on training dataf
pipeline_3.fit(X_train, y_train)

# Evaluate model on test data
accuracy_train_sv = pipeline_3.score(X_train, y_train)
accuracy_test_sv = pipeline_3.score(X_test, y_test)

print("SVM Training set score: {:.6f}".format(accuracy_train_sv)) 
print("SVM Test set score: {:.6f}".format(accuracy_test_sv))



In [ ]:
dump(svc, '/Users/nancy/Desktop/RS_projects/result/03_ML/01_train_model/01_featureV1/models//01_SVC_model_imbalance.joblib')

In [ ]:
# Save model: the same preprocessor already saved

dump(svc, '/Users/nancy/Desktop/RS_projects/result/03_ML/01_train_model/01_featureV1/models//01_SVC_model.joblib')